## Preamble

We have the same preamble in this Part as for the SQL notebooks in Parts 8-10. 

We will copy this section, **Preamble**, at the beginning of each of the notebooks to set up a clean version of the database at the beginning. In this notebook, where the content appears for the first time, we will explain what each of the cells is doing. From the next notebook, we will not include all the explanatory content. If you find at some point that you need a reminder of what the Preamble section is doing, you should come back to this notebook.

### Imports 

Import pandas as standard:

In [ ]:
import pandas as pd

Import the sql magic so that we can easily make calls on the database. The first call to 

In [ ]:
%%capture --no-display capture_output

%load_ext sql

And load the schema display magic so that we can see the entity relation diagrams for the tables.

In [ ]:
%%capture --no-display capture_output

%load_ext schemadisplay_magic

### Connect to the database

Construct the connection string using the variables set in the top cells.

We will use the variables `DB_USER` and `DB_PWD` to contain the name and password of the account that is being used to connect to the database.

For TM351, we have created an account with login ID `tm351` and password `tm351`.

In [ ]:
DB_USER='tm351'
DB_PWD='tm351'

To make the connection, we also need to state the database engine and the host and port on which the engine is listening.

In [ ]:
DB_ENGINE='postgresql' # tells ipython-sql that we will use postgreSQL as our database engine
DB_ADDR='localhost:5432' # the host and port on which the database engine is listening

And put them together to create the connection string:

In [ ]:
# Use urllib to escape any special characters in the password

import urllib

DB_CONNECTION = '{engine}://{user}:{pwd}@{addr}/{name}'.format(engine=DB_ENGINE,
                                                               user=DB_USER,
                                                               pwd=urllib.parse.quote_plus(DB_PWD),
                                                               addr=DB_ADDR,
                                                               name=DB_USER)


### Configure ipython-sql

By default, the sql magic returns queries as a `sql.run.ResultSet` object. We would prefer to return the values as a `pandas.DataFrame`:

In [ ]:
%%capture --no-display capture_output

# Set ipython-sql to return dataframes

%config SqlMagic.autopandas=True

By default, PostgreSQL returns results with the python `decimal` type, but *pandas* (or rather, `numpy`) prefers to work with `float`s:

In [ ]:
%%capture --no-display capture_output

# Set to return floats rather than decimals

import psycopg2

DEC2FLOAT = psycopg2.extensions.new_type(
    psycopg2.extensions.DECIMAL.values,
    'DEC2FLOAT',
    lambda value, curs: float(value) if value is not None else None)
psycopg2.extensions.register_type(DEC2FLOAT)